In [6]:

import torch
# The train split of client 1



1: ESOL(12)
2: FreeSolv(11)
3: Lipophilicity (13)
4: BACE (7)
5: BBBP (8)
6: ClinTox (6)
7: MUTAG (1)
8: PTC_MR (3)
9: PTC_MM (2)
10:  PTC_FM (4)
11: PTC_FR (5) Zu viele Datem: 1440
12: NCI109 (10)
13: NCI1 (9)
14: alchemy_full (15)
15: ZINC_full (16)
16: QM9 (14) (zu viele Daten? 130831)

In [8]:
import os
import sys

# from federatedscope.core.cmd_args import parse_args
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.auxiliaries.worker_builder import get_client_cls, get_server_cls
from federatedscope.core.configs.config import global_cfg
from federatedscope.core.fed_runner import FedRunner
from yacs.config import CfgNode
from rdkit.Chem.Draw import IPythonConsole

if os.environ.get('https_proxy'):
    del os.environ['https_proxy']
if os.environ.get('http_proxy'):
    del os.environ['http_proxy']

In [9]:
try:
    torch.multiprocessing.set_start_method('spawn', force=True)
except RuntimeError:
    pass

In [12]:
def train(client, lr):
    cfg_file = 'scripts/B-FHTL_exp_scripts/Graph-DC/isolated.yaml'
    cfg_client = 'scripts/B-FHTL_exp_scripts/Graph-DC/cfg_per_client.yaml'
    #'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client.yaml'

    init_cfg = global_cfg.clone()
    init_cfg.merge_from_file(cfg_file)

    # init_cfg.data.subdirectory = 'graph_dt_backup/processed'
    # init_cfg.merge_from_list(args.opts)
    init_cfg.data.client = client
    init_cfg.train.optimizer.lr = lr
    update_logger(init_cfg)

    # federated dataset might change the number of clients
    # thus, we allow the creation procedure of dataset to modify the global cfg object
    data, modified_cfg = get_data(config=init_cfg.clone())
    init_cfg.merge_from_other_cfg(modified_cfg)

    init_cfg.freeze()

    # allow different settings for different clients
    # cfg_client.merge_from_file(args.cfg_client)
    if cfg_client is None:
        cfg_client = None
    else:
        cfg_client = CfgNode.load_cfg(open(cfg_client, 'r')).clone()
    runner = FedRunner(data=data,
                   server_class=get_server_cls(init_cfg),
                   client_class=get_client_cls(init_cfg),
                   config=init_cfg.clone(),
                   client_config=cfg_client)
    _ = runner.run()

In [14]:
from torch import multiprocessing

clients = range(1, 13+1)
lrs = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]
num_trainings = 5

pool = multiprocessing.Pool(6)
processes = []

for client in clients:
    for lr in lrs:
        print(f"Client: {client},\tlr: {lr}")
        for i in range(num_trainings):
            setup_seed(i)
            processes.append(pool.apply_async(train, args=(client, lr)))
            print(f"training run: {i+1}")

result = [p.get() for p in processes]

Client: 1,	lr: 0.1
training run: 1
training run: 2
training run: 3
training run: 4
training run: 5
Client: 1,	lr: 0.05
training run: 1
training run: 2
training run: 3
training run: 4
training run: 5
Client: 1,	lr: 0.01
training run: 1
training run: 2
training run: 3
training run: 4
training run: 5
Client: 1,	lr: 0.005
training run: 1
training run: 2
training run: 3
training run: 4
training run: 5
Client: 1,	lr: 0.001
training run: 1
training run: 2
training run: 3
training run: 4
training run: 5
Client: 1,	lr: 0.0005
training run: 1
training run: 2
training run: 3
training run: 4
training run: 5
Client: 1,	lr: 0.0001
training run: 1
training run: 2
training run: 3
training run: 4
training run: 5
Client: 2,	lr: 0.1
training run: 1
training run: 2
training run: 3
training run: 4
training run: 5
Client: 2,	lr: 0.05
training run: 1
training run: 2
training run: 3
training run: 4
training run: 5
Client: 2,	lr: 0.01
training run: 1
training run: 2
training run: 3
training run: 4
training run

Process SpawnPoolWorker-928:
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'train' on <module '__main__' (built-in)>
Process SpawnPoolWorker-929:
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.9/multiprocessing/queues.py", line 368, in get
    

KeyboardInterrupt: 

Process SpawnPoolWorker-1068:
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'train' on <module '__main__' (built-in)>


In [ ]:
cfg_file = 'scripts/B-FHTL_exp_scripts/Graph-DT/isolated.yaml'
cfg_client = 'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_isolated.yaml'
#'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client.yaml'

init_cfg = global_cfg.clone()
init_cfg.merge_from_file(cfg_file)

# init_cfg.data.subdirectory = 'graph_dt_backup/processed'
# init_cfg.merge_from_list(args.opts)
init_cfg.data.client = 5
init_cfg.train.optimizer.lr = 0.01
update_logger(init_cfg)
setup_seed(init_cfg.seed)

# federated dataset might change the number of clients
# thus, we allow the creation procedure of dataset to modify the global cfg object
data, modified_cfg = get_data(config=init_cfg.clone())
init_cfg.merge_from_other_cfg(modified_cfg)

init_cfg.freeze()

# allow different settings for different clients
# cfg_client.merge_from_file(args.cfg_client)
if cfg_client is None:
    cfg_client = None
else:
    cfg_client = CfgNode.load_cfg(open(cfg_client, 'r')).clone()



In [ ]:
runner = FedRunner(data=data,
                   server_class=get_server_cls(init_cfg),
                   client_class=get_client_cls(init_cfg),
                   config=init_cfg.clone(),
                   client_config=cfg_client)
_ = runner.run()

In [ ]:
1

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)


In [ ]:
cfg_client['client_5']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_6']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_7']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_8']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_9']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_10']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_11']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_12']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_13']['eval']['metrics']=['mse']

In [ ]:
cl_list = ['client_' + str(i) for i in range(1,17)]

In [ ]:
cl_list

In [ ]:
cfg_client['client_13']['train']['local_update_steps']

In [ ]:
for el in cl_list:
    cfg_client[el]['train']['local_update_steps']=1

In [ ]:
cfg_client